# Advent of Code 2022

This solution (Jupyter lab 3.5.0; coconut 2.1.1 on python 3.10.8) by kannix68, @ 2022-12.  \
Using anaconda distro, conda v22.9.0, and coconut language.  \
Tested and run on MacOS v10.14.6 "Mojave".

Reddit Advent of Code [solution_megathreads - adventofcode](https://www.reddit.com/r/adventofcode/wiki/solution_megathreads#wiki_december_2021)

In [ ]:
import copy
import itertools
import logging
import math
import re
import sys
import time

from collections import defaultdict

import numpy as np
import pandas as pd

import pylib.aochelper as aoc
from pylib.aochelper import map_list as mapl
from pylib.aochelper import filter_list as filterl

f"Python version: {sys.version}" |> print
f"Version info: {sys.version_info}" |> print

log = aoc.getLogger(__name__)
f"Initial log-level={aoc.getLogLevelName(log.getEffectiveLevel())}." |> print

## Problem domain code

### --- Day 1: Calorie Counting ---

In [ ]:
"Day 1" |> print

tests = """
1000
2000
3000

4000

5000
6000

7000
8000
9000

10000
""".strip()

In [ ]:
def solve_day01pt1(inp):
  """Solve Day 1 part 1."""
  reindeers = inp.split("\n\n") |> fmap$( x -> x.split("\n") |> fmap$(int) )
  #reindeers |> print
  return reindeers |> map$(sum) |> max

In [ ]:
"Day 1 part 1" |> print
expected = 24_000
result = solve_day01pt1(tests)
aoc.assert_msg(f"test solve day 1 part 1 expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day01.in").strip()
out = solve_day01pt1(ins)
f"day 1 part 1 output: {out}" |> print

In [ ]:
def solve_day01pt2(inp):
  """Solve Day 1 part 2."""
  reindeers = inp.split("\n\n") |> fmap$( x -> x.split("\n") |> fmap$(int) )
  top3 = reindeers |> fmap$(sum) |> sorted$(reverse=True) |> .[0:3]
  return top3 |> sum

In [ ]:
"Day 1 part 2" |> print
expected = 45_000
result = solve_day01pt2(tests)
aoc.assert_msg(f"test solve day 1 part 1 expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day01pt2(ins)
f"day 1 part 2 output: {out}" |> print

### --- Day 2: Rock Paper Scissors ---

In [ ]:
tests = """
A Y
B X
C Z
""".strip()

In [ ]:
values = {'X': 1, 'Y': 2, 'Z': 3}
outcomes = {
  'A': {'X': 3, 'Y': 6, 'Z': 0},
  'B': {'X': 0, 'Y': 3, 'Z': 6},
  'C': {'X': 6, 'Y': 0, 'Z': 3},
}

def solve_day02pt1(inp):
  """Solve Day 2 part 1."""
  score = 0
  for line in inp.splitlines():
    [p1, p2] = line.split()
    #f"[p1, p2] = {[p1, p2]}" |> print
    round_score = values[p2]  # item value score
    round_score += outcomes[p1][p2]  # outcome score
    #f"adding round score {round_score}" |> print
    score += round_score
  return score

In [ ]:
"Day 2 part 1" |> print
expected = 15
result = solve_day02pt1(tests)
aoc.assert_msg(f"test solve day 2 part 1 expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day02.in").strip()
out = solve_day02pt1(ins)
f"day 2 part 1 output: {out}" |> print

In [ ]:
def solve_day02pt2(inp):
  """Solve Day 2 part 2."""
  outcome_tags = {'X': 0, 'Y': 3, 'Z': 6}  # who wins on pt 2
  outcomes_inv = {}
  for k, innerdict in outcomes.items():
    #outcomes_inv[k] = dict((v, k) for k, v in innerdict.items())  # keys and values swapped
    outcomes_inv[k] = {v: k for k, v in innerdict.items()}  # keys and values swapped
  #f"outcomes_inverse={outcomes_inv}" |> print
  score = 0
  for line in inp.splitlines():
    [p1, o2] = line.split()
    round_score = outcome_tags[o2]
    p2 = outcomes_inv[p1][round_score]  # my item (player 2)
    round_score += values[p2]  # outcome score
    #f"adding round score {round_score} from item={p2}" |> print
    score += round_score
  return score

In [ ]:
"Day 2 part 2" |> print
expected = 12
result = solve_day02pt2(tests)
aoc.assert_msg(f"test solve day 2 part 2 expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day02pt2(ins)
f"day 2 part 2 output: {out}" |> print

### --- Day 3: Rucksack Reorganization ---

In [ ]:
tests = """
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
""".strip()

In [ ]:
def split_str_halfs(s):
  """Split a string in two of equel length"""
  pos = len(s)//2
  return [s[:pos], s[pos:]]

def solve_day03pt1(inp):
  """Solve Day 3 part 1."""
  score = 0
  ofst_lower = ord("a") - 1
  ofst_upper = ord("A") - 27
  for line in inp.splitlines():
    sh1, sh2 = split_str_halfs(line)
    #[sh1, sh2] |> print
    freqh1, freqh2 = [defaultdict(int), defaultdict(int)]
    for c in sh1:
      freqh1[c] += 1
    for c in sh2:
      freqh2[c] += 1
    isct = set(freqh1.keys()).intersection(set(freqh2.keys()))
    assert len(isct) == 1
    isct = list(isct)[0]
    if isct.isupper():
      prio = ord(isct) - ofst_upper
    else:
      prio = ord(isct) - ofst_lower
    score += prio
    #f"set-intersection={isct}, prio={prio}" |> print
  return score

In [ ]:
"Day 3 part 1" |> print
expected = 157
result = solve_day03pt1(tests)
aoc.assert_msg(f"test solve day 3 part 1 expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str("./in/day03.in").strip()
out = solve_day03pt1(ins)
f"day 3 part 1 output: {out}" |> print

In [ ]:
def solve_day03pt2(inp):
  """Solve Day 3 part 2."""
  score = 0
  ofst_lower = ord("a") - 1
  ofst_upper = ord("A") - 27
  grp = []
  for idx, line in enumerate(inp.splitlines()):
    grp.append(line)
    if len(grp) == 3:
      tgtset = set()
      for idx, memb in enumerate(grp):
        freqs = defaultdict(int)
        for c in memb:
          freqs[c] += 1
        if idx==0:
          tgtset = set(freqs.keys())
        else:
          tgtset = tgtset.intersection(freqs.keys())
      grp = []
      assert len(tgtset) == 1
      un = list(tgtset)[0]
      if un.isupper():
        prio = ord(un) - ofst_upper
      else:
        prio = ord(un) - ofst_lower
      #f"grp set-intersection={tgtset}, prio={prio}" |> print
      score += prio
  assert idx > 0 
  assert (idx+1) % 3 == 0
  return score

In [ ]:
"Day 3 part 2" |> print
expected = 70
result = solve_day03pt2(tests)
aoc.assert_msg(f"test solve day 3 part 2 expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day03pt2(ins)
f"day 3 part 2 output: {out}" |> print

### --- Day 4: Camp Cleanup ---

In [ ]:
day = "Day 4"
symb = "day04"
part = "part 1"
tests = """
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
""".strip()

In [ ]:
def has_fully_containment(e1, e2):
  return (e1[0] <= e2[0] and e1[1] >= e2[1]) or (e2[0] <= e1[0] and e2[1] >= e1[1])

def solve_day04pt1(inp):
  """Solve Day 4 part 1."""
  score = 0
  #pairs = []
  for line in inp.splitlines():
    elves = line.split(",")
    elves = list(map(lambda it: list(map(int, it.split("-"))), elves))
    if has_fully_containment(elves[0], elves[1]):
      #f"{elves} has fully containment" |> print
      score += 1
  #pairs |> print
  return score

In [ ]:
f"{day} {part}" |> print
expected = 2
result = solve_day04pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day04pt1(ins)
f"{day} {part} output: {out}" |> print

In [ ]:
def has_overlap(e1, e2):
  return (e1[1] >= e2[0] and e1[1] <= e2[1]) or (e2[1] >= e1[0] and e2[1] <= e1[1])

def solve_day04pt2(inp):
  """Solve Day 4 part 2."""
  score = 0
  for line in inp.splitlines():
    elves = line.split(",")
    elves = list(map(lambda it: list(map(int, it.split("-"))), elves))
    if has_overlap(elves[0], elves[1]):
      #f"{elves} has overlap" |> print
      score += 1
  return score

In [ ]:
part = "part 2"
expected = 4
f"{day} {part}" |> print
result = solve_day04pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day04pt2(ins)
f"{day} {part} output: {out}" |> print

### --- Day 5: Supply Stacks ---

In [ ]:
day = "Day 5"
symb = "day05"
part = "part 1"
tests = """
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
"""

In [ ]:
def pop_n(l, n):
  """Pop n members from top of stack (list), return popped head and altered stack."""
  popped, popped_lst = [ l[-n:], l[0:len(l)-n] ]
  return [popped, popped_lst]

def push_n(l, data):
  """Push data (list) members to top of stack (list), return altered stack."""
  for elem in data:
    l.append(elem)
  return l

def transpose_lol(lol):
  """Transpose a list-of-lists."""
  # short circuits at shortest nested list if table is jagged:
  return list(map(list, zip(*lol)))

def get_stacks(rows):
  """Take parsed rows from input string and pprocess into target data structure list-of-tacks."""
  stacks = transpose_lol(rows)
  stacks = mapl(lambda it: list(reversed(it)), stacks)
  for stack in stacks:
    while stack[-1] == "":
      stack.pop()
  return stacks

def play_stack_cmds(stacks, cmds):
  """Play the commands on stacks data."""
  for cmd in cmds:
    anz, frm, to = cmd
    hd, stacks[frm-1] = pop_n(stacks[frm-1], anz)
    stacks[to-1] = push_n(stacks[to-1], reversed(hd))
  return stacks

In [ ]:
def solve_day05pt1(inp):
  """Solve Day 5 part 1."""
  chunk_size = 4
  stacks = []
  rows = []
  cmds = []
  #f"inp=\n{inp}" |> print
  for line in inp.split("\n"):
    #f"line=>{line}< len={len(line)}" |> print
    line += " "
    if "[" in line:
      llen = len(line)
      row = [ line[i:i+chunk_size] for i in range(0, llen, chunk_size) ]
      #f"found row1={row}" |> print
      row = mapl(lambda it: it.replace("[", "").replace("]", "").strip(), row)
      #f"found row2={row}" |> print
      rows.append(row)
    elif line.strip() == "":
      #"empty line found" |> print
      continue
    elif "move" in line:
      m = re.search(r"^move (\d+) from (\d+) to (\d+)", line)
      l = mapl(int, [m.group(1), m.group(2), m.group(3)])
      cmds.append(l)
    else:
      #f"unparsed line=>{line}<" |> print
      continue
  stacks = get_stacks(rows)
  #f"initial_stacks=\n{stacks}" |> print
  #f"  cmds=\n{cmds}" |> print
  stacks = play_stack_cmds(stacks, cmds)
  result = "".join([stack.pop() for stack in stacks])
  #f"result={result}" |> print
  return result

In [ ]:
f"{day} {part}" |> print
expected = "CMZ"
result = solve_day05pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in")
out = solve_day05pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def play_stack_cmds_pt2(stacks, cmds):
  """Play the commands on stacks data, modified for part 2."""
  for cmd in cmds:
    anz, frm, to = cmd
    hd, stacks[frm-1] = pop_n(stacks[frm-1], anz)
    stacks[to-1] = push_n(stacks[to-1], hd)  # changed, not reversed
  return stacks

In [ ]:
def solve_day05pt2(inp):
  """Solve Day 5 part 2."""
  chunk_size = 4
  stacks = []
  rows = []
  cmds = []
  for line in inp.split("\n"):
    line += " "
    if "[" in line:
      llen = len(line)
      row = [ line[i:i+chunk_size] for i in range(0, llen, chunk_size) ]
      row = mapl(lambda it: it.replace("[", "").replace("]", "").strip(), row)
      rows.append(row)
    elif line.strip() == "":
      continue
    elif "move" in line:
      m = re.search(r"^move (\d+) from (\d+) to (\d+)", line)
      l = mapl(int, [m.group(1), m.group(2), m.group(3)])
      cmds.append(l)
    else:
      continue
  stacks = get_stacks(rows)
  stacks = play_stack_cmds_pt2(stacks, cmds)  # changed, pt2
  result = "".join([stack.pop() for stack in stacks])
  return result

In [ ]:
part = "part 2"
expected = "MCD"
f"{day} {part}" |> print
result = solve_day05pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day05pt2(ins)
f"{day} {part} output: {out}" |> print

### --- Day 6: Tuning Trouble ---

In [ ]:
day = "Day 6"
symb = "day06"
part = "part 1"
tests = """
mjqjpqmgbljsphdztnvjfqwrcgsmlb
""".strip()  # 7
test2 = "bvwbjplbgvbhsrlpgdmjqwftvncz"  # 5
test3 = "nppdvjthqldpwncqszvftbrmjlhg"  # 6
test4 = "nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg"  # 10
test5 = "zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw"  # 11

In [ ]:
def solve_day06(inp, pkglen=4):
  """Solve Day 6 part 1."""
  pkgct = pkglen - 1
  result = -1
  for idx, c in enumerate(inp):
    if idx < pkgct:
      continue
    l = list(inp[idx-pkgct:idx+1])
    #l |> print
    st = set(l)
    if len(st) == pkglen:
      result = idx+1
      break
  return result

In [ ]:
f"{day} {part}" |> print
expected = 7
result = solve_day06(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

expected = 5; result = solve_day06(test2)
aoc.assert_msg(f"test2 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 6; result = solve_day06(test3)
aoc.assert_msg(f"test3 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 10; result = solve_day06(test4)
aoc.assert_msg(f"test4 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 11; result = solve_day06(test5)
aoc.assert_msg(f"test5 solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in")
ins = ins.splitlines()[0]
out = solve_day06(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
part = "part 2"
expected = 19
f"{day} {part}" |> print
result = solve_day06(tests, pkglen=14)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

expected = 23; result = solve_day06(test2, pkglen=14)
aoc.assert_msg(f"test2 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 23; result = solve_day06(test3, pkglen=14)
aoc.assert_msg(f"test3 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 29; result = solve_day06(test4, pkglen=14)
aoc.assert_msg(f"test4 solve {day} {part} expected={expected} result={result}", result == expected)

expected = 26; result = solve_day06(test5, pkglen=14)
aoc.assert_msg(f"test5 solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day06(ins, pkglen=14)
f"{day} {part} output: {out}" |> print

### --- Day 7: No Space Left On Device ---

In [ ]:
day = "Day 7"
symb = "day07"
part = "part 1"
tests = """
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
""".strip()

In [ ]:
def get_dirsizes(inp):
  from pathlib import Path
  from collections import Counter

  dirsizes = Counter()
  currdir = top = Path('/')

  for line in inp.splitlines():
    args = line.strip().split()
    if args[0] == '$':
      if args[1] == 'cd':
        if args[2] == '/':
          currdir = top
        elif args[2] == '..':
          currdir = currdir.parent
        else:
          currdir /= args[2]
    elif args[0] != 'dir':
      #dirsizes[currdir] += (size := int(args[0]))
      size = int(args[0])
      dirsizes[currdir] += size
      for p in currdir.parents:
        dirsizes[p] += size
  return dirsizes

def solve_day07pt1(inp):
  #dirsizes = get_dirsizes(inp)
  #sizelist = sorted(dirsizes.values())
  sizelist = get_dirsizes(inp).values()
  return sum(n for n in sizelist if n <= 100_000)

In [ ]:
f"{day} {part}" |> print
expected = 95437
result = solve_day07pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day07pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def solve_day07pt2(inp):
  #print('P2 =', sizelist[bisect(sizelist, dirsizes[top] - 4000_0000)])
  sizelist = get_dirsizes(inp).values()
  totsize = max(sizelist)
  sizelist = filterl(lambda it: it >= totsize - 4000_0000, sizelist)
  return min(sizelist)

In [ ]:
part = "part 2"
f"{day} {part}" |> print
expected = 24933642
result = solve_day07pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day07pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 8: Treetop Tree House ---

In [ ]:
day = "Day 8"
symb = "day08"
part = "part 1"
tests = """
30373
25512
65332
33549
35390
""".strip()

In [ ]:
def visible_from(dirctn, lol, x, y):
  is_visible = False
  val = lol[y][x]
  if dirctn == 'W':
    row = lol[y][0:x]
  elif dirctn == 'E':
    row = lol[y][x+1:]
    pass
  elif dirctn == 'N':
    row = transpose_lol(lol)[x][0:y]
  elif dirctn == 'S':
    row = transpose_lol(lol)[x][y+1:]
  is_visible = val > max(row)
  #f"visible_from({dirctn}, lol, {x}, {y}) val={val} vis?={is_visible} row={row}" |> log.debug
  return is_visible

def solve_day08pt1(inp):
  result = 0
  lol = mapl(lambda it: mapl(lambda inr: int(inr), list(it)), inp.splitlines())
  #f"lol={lol}" |> log.debug
  l, w = len(lol), len(lol[0])
  #circum = 2*l+2*w-4
  #f"grid l={l}, w={w} boundary={circum}" |> log.debug
  f"grid l={l}, w={w}" |> log.debug
  #lolvis = [ [0]*w ]*l # NO! identical rows!
  #lolvis = [[0 for x in range(w)] for y in range(l)]
  for y, row in enumerate(lol):
    for x, cell in enumerate(row):
      #f"pt({x},{y})" |> print
      if x in [0, w-1]:
        result +=1
        #f"  setX-pt({x},{y})" |> print
        #lolvis[y][x] = 1
        #f"  lolvis={lolvis}" |> log.debug
      elif y in [0, l-1]:
        result +=1
        #f"  setY-pt({x},{y})" |> print
        #lolvis[y][x] = 1
        #f"  lolvis={lolvis}" |> log.debug
      elif visible_from("W", lol, x, y) or visible_from("E", lol, x, y) \
        or visible_from("S", lol, x, y) or visible_from("N", lol, x, y):
        result +=1
        #lolvis[y][x] = 1
        #vn = visible_from("S", lol, x, y)
        #f"    vN={vn}" |> log.debug
  #f"lolvis={lolvis}" |> log.debug
  return result

In [ ]:
f"{day} {part}" |> print
log.setLevel(logging.DEBUG)
expected = 21
result = solve_day08pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day08pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def get_viewdist(dirctn, lol, x, y):
  val = lol[y][x]
  if dirctn == 'W':
    row = lol[y][0:x]
    row = list(reversed(row))
  elif dirctn == 'E':
    row = lol[y][x+1:]
    pass
  elif dirctn == 'N':
    row = transpose_lol(lol)[x][0:y]
    row = list(reversed(row))
  elif dirctn == 'S':
    row = transpose_lol(lol)[x][y+1:]
  #f"get_viewdist({dirctn}, lol, {x}, {y}): val={val} row={row}" |> print
  mx = 0
  score = 0
  holds = 0
  for i in row:
    if i >= val: # blocked immediately higher than current tree house
      score += 1
      if holds > 0:
        score += holds
      break
    elif i >= mx:
      mx = i
      score += 1
      if holds > 0:
        score += holds
        holds = 0
    else:
      holds += 1
  #f"  score={score}" |> print
  return score

def get_scenic_score(lol, x, y):
  #if x == 0 or y == 0 or y == len(lol)-1 or x == len(lol[0])-1:
  #  return 0  # short circuit on boundary cells
  scores = []
  scores.append(get_viewdist("W", lol, x, y))
  scores.append(get_viewdist("E", lol, x, y))
  scores.append(get_viewdist("N", lol, x, y))
  scores.append(get_viewdist("S", lol, x, y))
  #f"get_scenic_score(lol, {x}, {y}) scores={scores}" |> log.debug
  #rc = scores |> reduce$(sum)
  rc = reduce(lambda r,l: r*l, scores)
  #f"  rc={rc}" |> log.debug
  return rc

def get_lol_from_str(inp):
  return mapl(lambda it: mapl(lambda inr: int(inr), list(it)), inp.splitlines())

def test_scenic_score(inp, x, y):
  f"test_scenic_score(s, {x}, {y})" |> log.debug
  return get_scenic_score(get_lol_from_str(inp), x, y)

def solve_day08pt2(inp):
  result = 0
  lol = mapl(lambda it: mapl(lambda inr: int(inr), list(it)), inp.splitlines())
  l, w = len(lol), len(lol[0])
  f"grid l={l}, w={w}" |> log.debug
  for y, row in enumerate(lol):
    for x, cell in enumerate(row):
      score = get_scenic_score(lol, x, y)
      if score > result:
        f"found score={score} @pt({x},{y}) val={lol[y][x]}" |> log.debug
        result = score
  return result

In [ ]:
part = "part 2"
f"{day} {part}" |> print
expected = 4
result = test_scenic_score(tests, 2, 1) # one point, val=5
aoc.assert_msg(f"test get_scenic_score 1 {day} {part} expected={expected} result={result}", result == expected)

expected = 8
result = test_scenic_score(tests, 2, 3) # oone point, val=5
aoc.assert_msg(f"test get_scenic_score 2 {day} {part} expected={expected} result={result}", result == expected)

expected = 8
result = solve_day08pt2(tests) # all points
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

# TODO: FIXME: solve_day08pt2
assert False, "currently FAILS output too low"
out = solve_day08pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 9: Rope Bridge ---

In [ ]:
day = "Day 9"
symb = "day09"
part = "part 1"
tests = """
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
""".strip()

In [ ]:
def solve_day09pt1(inp):
  result = 0
  lol = mapl(lambda it: it.split(), inp.splitlines())
  #lol |> log.debug
  h = t = tuple([0, 0])
  vec = {
    "R": tuple([1, 0]), "L": tuple([-1, 0]),
    "U": tuple([0, 1]), "D": tuple([0, -1])
  }
  ct = 0
  posns = set()
  #f"ct={ct}: H={h}, T={t}" |> log.debug
  for cmd in lol:
    for times in range(int(cmd[1])):
      ct += 1
      v = vec[cmd[0]]
      h = tuple([h[0]+v[0], h[1]+v[1]])
      manhdist = [h[0]-t[0], h[1]-t[1]]
      md = abs(h[0]-t[0]) + abs(h[1]-t[1])
      dx, dy = np.sign(h[0]-t[0]), np.sign(h[1]-t[1])
      bd = dx != 0 and dy != 0
      #f"cmd={cmd} md={md} dx,dy={dx},{dy} bd={bd}" |> log.debug
      if bd == False and md > 1: # ortho
        #f"    mv-ortho" |> log.debug
        t = tuple([t[0]+dx, t[1]+dy])
        pass
      elif bd == True and md > 2:
        #f"    mv-diag" |> log.debug
        t = tuple([t[0]+dx, t[1]+dy])
      else:
        #f"    nop-mv" |> log.debug
        pass
      posns.add(t)
      #f"ct={ct}: H={h}, T={t}; md={md} >{manhdist}; posns={posns}" |> log.debug
  return len(posns)

In [ ]:
f"{day} {part}" |> print
log.setLevel(logging.DEBUG)
expected = 13
result = solve_day09pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day09pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def repr_grid_nodes(nodes):
  """Represent a grid from nodes koordinates, returns string."""
  x_mx = max(mapl(lambda it: it[0], nodes))
  y_mx = max(mapl(lambda it: it[1], nodes))
  x_min = min(mapl(lambda it: it[0], nodes))
  y_min = min(mapl(lambda it: it[1], nodes))
  if x_min < 0:
    x_ofst = -x_min
  else:
    x_ofst = 0
  if y_min < 0:
    y_ofst = -y_min
  else:
    y_ofst = 0

  f"repr_grid_nodes xmx={x_mx},ymx={y_mx}, nds={nodes}" |> log.debug
  repr_los = []
  for y in range(y_ofst+y_mx+1):
    repr_los.append("." * (x_ofst+x_mx+1))
  for i in reversed(range(len(nodes))):
    nd_x, nd_y = nodes[i]
    trg = list(repr_los[y_ofst+nd_y])
    trg[x_ofst+nd_x] = f"{i}"
    repr_los[y_ofst+nd_y] = "".join(trg)
  return "\n".join(reversed(repr_los))

def follow_node(i, nodes, dr):
  if i == 0:
    vec = {
      "R": tuple([1, 0]), "L": tuple([-1, 0]),
      "U": tuple([0, 1]), "D": tuple([0, -1])
    }
    v = vec[dr]
    nodes[i] = tuple([nodes[i][0]+v[0], nodes[i][1]+v[1]])
    f"newpos-hd={nodes[i]}" |> log.debug
  else:
    #headknot, leadknot, thisknot = nodes[0], nodes[i-1], nodes[i]
    ldknt, crknt = nodes[i-1], nodes[i]
    manhdist = [ldknt[0]-crknt[0], ldknt[1]-crknt[1]]
    md = abs(ldknt[0]-crknt[0]) + abs(ldknt[1]-crknt[1])
    dx, dy = np.sign(ldknt[0]-crknt[0]), np.sign(ldknt[1]-crknt[1])
    bd = dx != 0 and dy != 0
    if bd == False and md > 1: # ortho
      t = tuple([crknt[0]+dx, crknt[1]+dy])
      nodes[i] = t
    elif bd == True and md > 2:
      t = tuple([crknt[0]+dx, crknt[1]+dy])
      nodes[i] = t
    else:
      pass
  return nodes

def solve_day09pt2(inp):
  result = 0
  lol = mapl(lambda it: it.split(), inp.splitlines())
  #lol |> log.info
  h = t = tuple([0, 0])
  nodes = [tuple([0, 0]) for i in range(10)]
  ct = 0
  #posns = [set() for i in range(10)]
  posns = set()
  f"ct={ct}: H={nodes[0]}" |> log.debug
  for cmd in lol:
    for times in range(int(cmd[1])):
      ct += 1
      log.debug(f"ct={ct} repr=\n{repr_grid_nodes(nodes)}")
      ct += 1
      for i in range(10):
        nodes = follow_node(i, nodes, cmd[0])
        posns.add(nodes[9])
      #f"ct={ct}: H={h}, T={t}; md={md} >{manhdist}; posns={posns}" |> log.debug
      #f"ct={ct}: posns={mapl(lambda it: len(it9, posns)}" |> log.debug()
  print(f"ct={ct} repr=\n{repr_grid_nodes(nodes)}")
  return len(posns)

In [ ]:
part = "part 2"
f"{day} {part}" |> print
log.setLevel(logging.INFO)
expected = 1
result = solve_day09pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
test2 = """
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
""".strip()
expected = 36
result = solve_day09pt2(test2)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
out = solve_day09pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 10: Cathode-Ray Tube ---

In [ ]:
day = "Day 10"
symb = "day10"
part = "part 1"
tests = """
noop
addx 3
addx -5
""".strip()

In [ ]:
test2 = """
addx 15
addx -11
addx 6
addx -3
addx 5
addx -1
addx -8
addx 13
addx 4
noop
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx -35
addx 1
addx 24
addx -19
addx 1
addx 16
addx -11
noop
noop
addx 21
addx -15
noop
noop
addx -3
addx 9
addx 1
addx -3
addx 8
addx 1
addx 5
noop
noop
noop
noop
noop
addx -36
noop
addx 1
addx 7
noop
noop
noop
addx 2
addx 6
noop
noop
noop
noop
noop
addx 1
noop
noop
addx 7
addx 1
noop
addx -13
addx 13
addx 7
noop
addx 1
addx -33
noop
noop
noop
addx 2
noop
noop
noop
addx 8
noop
addx -1
addx 2
addx 1
noop
addx 17
addx -9
addx 1
addx 1
addx -3
addx 11
noop
noop
addx 1
noop
addx 1
noop
noop
addx -13
addx -19
addx 1
addx 3
addx 26
addx -30
addx 12
addx -1
addx 3
addx 1
noop
noop
noop
addx -9
addx 18
addx 1
addx 2
noop
noop
addx 9
noop
noop
noop
addx -1
addx 2
addx -37
addx 1
addx 3
noop
addx 15
addx -21
addx 22
addx -6
addx 1
noop
addx 2
addx 1
noop
addx -10
noop
noop
addx 20
addx 1
addx 2
addx 2
addx -6
addx -11
noop
noop
noop
""".strip()

In [ ]:
def parse_prog(s):
  instrctns = []
  for line in s.splitlines():
    instrctn = line.split()
    f"instrctn={instrctn}" |> log.trace
    instrctns.append(instrctn)
  f"prog-length is {len(instrctns)}" |> log.debug
  return instrctns

def run_cpu(prog):
  checkpts= [20, 60, 100, 140, 180, 220]
  sigs = []
  cycle = 0
  reg = 1
  for instrctn in prog:
    cycle += 1
    cmd = instrctn[0]
    f"cycle={cycle}, reg={reg}, instrctn={instrctn}" |> log.trace
    if cycle in checkpts:
      f"checkpt cycle={cycle} reg={reg} sig-str={cycle*reg}" |> log.debug
      sigs.append(cycle*reg)
    if cmd == "noop":
      pass
    elif cmd == "addx":
      cycle += 1
      if cycle in checkpts:
        f"checkpt cycle={cycle} reg={reg} sig-str={cycle*reg}" |> log.trace
        sigs.append(cycle*reg)
      reg += int(instrctn[1])
    else:
      assert False, f"unknown instruction {cmd}"
  f"fin-cycle={cycle}, reg={reg}, instrctn={instrctn}" |> log.debug
  #f"sum-sig-strenths={sum(sigs)}" |> log.info
  return sum(sigs)

def solve_day10pt1(inp):
  return run_cpu(parse_prog(inp))

In [ ]:
f"{day} {part}" |> print
log.setLevel(logging.INFO)
expected = 13140
result = solve_day10pt1(test2)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day10pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def run_crt(prog):
  cycle = 0
  reg = 1
  crt = ""
  for instrctn in prog:
    if cycle % 40 == 0:
      crt += "\n"
    if abs((cycle % 40) - reg) < 2:
      crt += "#"
    else:
      crt += "."
    cycle += 1
    cmd = instrctn[0]
    f"cycle={cycle}, reg={reg}, instrctn={instrctn}" |> log.debug
    if cmd == "noop":
      pass
    elif cmd == "addx":
      if cycle % 40 == 0:
        crt += "\n"
      if abs((cycle % 40) - reg) < 2:
        crt += "#"
      else:
        crt += "."
      cycle += 1
      f"cycle={cycle}, reg={reg}, instrctn={instrctn}" |> log.debug
      reg += int(instrctn[1])
    else:
      assert False, f"unknown instruction {cmd}"
  f"fin-cycle={cycle}, reg={reg}, instrctn={instrctn}" |> log.info
  #f"sum-sig-strenths={sum(sigs)}" |> log.info
  return crt

def solve_day10pt2(inp):
  return run_crt(parse_prog(inp))

In [ ]:
part = "part2"
f"{day} {part}" |> print
log.setLevel(logging.INFO)
expected = 13140
result = solve_day10pt2(test2)
print(f"Show test solve {day} {part} expected={expected} result={result}")

In [ ]:
out = solve_day10pt2(ins)
f"Show {day} {part} result: {out}" |> print

### --- Day 11: Monkey in the Middle ---

In [ ]:
day = "Day 11"
symb = "day11"
part = "part 1"
tests = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
""".strip()

In [ ]:
#Monkey 0:
#  Starting items: 79, 98
#  Operation: new = old * 19
#  Test: divisible by 23
#    If true: throw to monkey 2
#    If false: throw to monkey 3
def parse_setup(s):
  monkeys = []
  for line in s.splitlines():
    #f"line: {line}" |> log.trace
    if m := re.search(r"^Monkey (.*):", line):
      monkey = {}
      monkey["id"] = int(m.group(1))
      f"found monkey {monkey['id']}" |> log.debug
      monkeys.append(monkey)
    elif m := re.search(r"^  Starting items: (.*)", line):
      monkey["items"] = mapl(int, m.group(1).split(", "))
    elif m := re.search(r"^  Operation: (.*)", line):
      #monkey["op-str"] = m.group(1)
      tmps = m.group(1).replace("new = old ", "")
      #monkey["op-op"], monkey["op-arg"] = tmps.split()
      monkey["op"] = tmps.split()
      #monkey["op"][1] = int(monkey["op"][1])
    elif m := re.search(r"^  Test: (.*)", line):
      tsts = {}
      tmps = m.group(1)
      assert .startswith("divisible by "), "test op"
      monkey["test"] = tsts
      #tsts["cond"] = m.group(1)
      tsts["cond-mod"] = int(tmps.replace("divisible by ", ""))
    elif m := re.search(r"^    If (.*): throw to monkey (.*)", line):
      tsts[m.group(1)] = int(m.group(2))
    elif line.strip() == "":
      pass
    else:
      f"unparsed line {line}" |> log.info
    monkey["insp_num"] = 0
    f"monkey={monkey}" |> log.trace
  f"monkeys={monkeys}" |> log.debug
  return monkeys

def do_round(gmdat):
  newdat = copy.deepcopy(gmdat)
  for monkey in newdat:
    id = monkey["id"]
    while len(monkey["items"]) > 0:
      val = monkey["items"][0]
      monkey["insp_num"] += 1
      monkey["items"] = monkey["items"][1:]
      op_op, op_arg = monkey["op"]
      if op_op == "*":
        if op_arg == "old":
          val *= val
        else:
          val *= int(op_arg)
      elif op_op == "+":
        val += int(op_arg)
      else:
        assert False, f"unparsed operator {op_op}"
      val = val // 3
      tst = monkey["test"]
      if val % tst["cond-mod"] == 0:
        trg_id = tst["true"]
      else:
        trg_id = tst["false"]
      newdat[trg_id]["items"].append(val)
      #log.trace(f"mkd={id} post-inspct-val={val} thrown to mnk={trg_id}")
    assert len(newdat[id]["items"]) == 0
  #for monkey in newdat:
  #  log.debug(f"mnk={monkey['id']} items={monkey['items']}")
  return newdat

def repr_gamedat(gmdat, show_plain=True):
  for monkey in gmdat:
    if show_plain:
      log.info(f"mnk={monkey['id']}, inspctd#={monkey['insp_num']}, items={monkey['items']}")
    else:
      nits = []
      for item in monkey['items']:
        if item < 1_000_000_000:
          nits.append(f"v{item}")
        else :
          nits.append(f"L{len(str(item))}")
      log.info(f"mnk={monkey['id']}, inspctd#={monkey['insp_num']}, items={nits}")

def solve_day11pt1(inp, rounds=20):
  gmdat = parse_setup(inp)
  for idx in range(10_002):
    if idx == rounds:
      break
    gmdat = do_round(gmdat)
    log.debug(f"rd={idx+1}::")
    for monkey in gmdat:
      log.debug(f"mnk={monkey['id']}, inspctd#={monkey['insp_num']}, items={monkey['items']}")
  #l = sorted(mapl(lamda it: it["insp_num"], gmdat))[0:2]
  l = mapl(lambda it: it["insp_num"], gmdat)
  l = reversed(sorted(l))[0:2]
  mb = reduce(lambda a,b: a*b, l)
  log.info(f"after rd={idx}::")
  log.info(repr_gamedat(gmdat))
  log.info(f"mnk-biz={mb} from {l}")
  return mb

In [ ]:
f"{day} {part}" |> print
log.setLevel(logging.WARN)
expected = 10605
result = solve_day11pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
out = solve_day11pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def do_round_v2(gmdat, lcm):
  newdat = copy.deepcopy(gmdat)
  for monkey in newdat:
    id = monkey["id"]
    while len(monkey["items"]) > 0:
      val = monkey["items"][0]
      monkey["insp_num"] += 1
      monkey["items"] = monkey["items"][1:]
      op_op, op_arg = monkey["op"]
      if op_op == "*":
        if op_arg == "old":
          val *= val
        else:
          val *= int(op_arg)
      elif op_op == "+":
        val += int(op_arg)
      else:
        assert False, f"unparsed operator {op_op}"
      # "In short: Calculate the remainder of the target worry level with the L.C.M of all the divisors."
      val = val % lcm  # !!!
      tst = monkey["test"]
      if val % tst["cond-mod"] == 0:
        trg_id = tst["true"]
      else:
        trg_id = tst["false"]
      newdat[trg_id]["items"].append(val)
    assert len(newdat[id]["items"]) == 0
  return newdat

def solve_day11pt2(inp, rounds=20):
  gmdat = parse_setup(inp)
  mults = []
  for monkey in gmdat:
    mults.append( monkey["test"]["cond-mod"] )
  lcm = math.lcm(*mults)
  log.info(f"lcm={lcm} from mults={mults}")
  for idx in range(10_002):
    if idx == rounds:
      break
    gmdat = do_round_v2(gmdat, lcm)
    log.debug(f"rd={idx+1}::")
    for monkey in gmdat:
      log.debug(f"mnk={monkey['id']}, inspctd#={monkey['insp_num']}, items={monkey['items']}")
  l = mapl(lambda it: it["insp_num"], gmdat)
  l = reversed(sorted(l))[0:2]
  mb = reduce(lambda a,b: a*b, l)
  log.info(f"after rd={idx}::")
  log.info(repr_gamedat(gmdat))
  log.info(f"mnk-biz={mb} from {l}")
  return mb

In [ ]:
part = "part 2"
expected = 10197
result = solve_day11pt2(tests, rounds=20)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
rounds = 1000
#sys.set_int_max_str_digits(1_000_000) # default 4300
tms = int( time.time() )
result = solve_day11pt2(tests, rounds=rounds)
tmd = int( time.time() ) - tms
print(f"rounds={rounds} res={result}, took {tmd} secs")

In [ ]:
rounds = 10_000
tms = int( time.time() )
result = solve_day11pt2(tests, rounds=rounds)
tmd = int( time.time() ) - tms
print(f"rounds={rounds} res={result}, took {tmd} secs")

In [ ]:
rounds = 10_000
tms = int( time.time() )
result = solve_day11pt2(ins, rounds=rounds)
tmd = int( time.time() ) - tms
print(f"rounds={rounds} res={result}, took {tmd} secs")

### --- Day 12: Hill Climbing Algorithm ---

In [ ]:
day = "Day 12"
symb = "day12"
part = "part 1"
tests = """
Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi
""".strip()

In [ ]:
import networkx as nx  # Graph theory module

def solve_day12pt1(inp):
  """Day 12 pt1 solution."""
  ofst = ord("a")
  g = nx.DiGraph()
  lines = inp.splitlines()
  posns = set()
  heights = {}
  for idxh, ln in enumerate(lines):
    if idxh == 0:
      w = len(ln)
      h = len(lines)
    for idxw, node in enumerate(list(ln)):
      if node == "S":
        height = ord("a") - ofst
      elif node == "E":
        height = ord("z") - ofst
      else:
        height = ord(node) - ofst
      pos = tuple([idxw, idxh])
      heights[pos] = height
      g.add_node(pos, height=height, cell=node)
      posns.add(pos)
      if node == "S":
        startpos = pos
        log.debug(f"nd-start pos={pos}")
      elif node == "E":
        endpos = pos
        log.debug(f"nd-end pos={pos}")
      pos_ab = tuple([idxw, idxh-1])
      pos_lf = tuple([idxw-1, idxh])
      if pos_ab in posns:
        h1 = heights[pos_ab]
        h2 = height
        log.trace(f"add-edges above {pos_ab}[{h1}] <-> {pos}[{h2}]")
        if h2 <= h1 + 1:
          log.trace(f"add-edge above {pos_ab} -> {pos}")
          g.add_edge(pos_ab, pos)
        if h1 <= h2 + 1:
          log.trace(f"add-edge above {pos_ab} <- {pos}")
          g.add_edge(pos, pos_ab)
      if pos_lf in posns:
        h1 = heights[pos_lf]
        h2 = height
        log.trace(f"add-edges above {pos_lf}[{h1}] <-> {pos}[{h2}]")
        if h2 <= h1 + 1:
          log.trace(f"add-edge above {pos_lf} -> {pos}")
          g.add_edge(pos_lf, pos)
        if h1 <= h2 + 1:
          log.trace(f"add-edge above {pos_lf} <- {pos}")
          g.add_edge(pos, pos_lf)
  log.trace(f"graph={g}")
  pth = nx.shortest_path(g, source=startpos, target=endpos)
  log.debug(f"path={pth}")
  return len(pth)-1  # dont count start node

In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 31
result = solve_day12pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
log.setLevel(logging.INFO)
out = solve_day12pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def solve_day12pt2(inp):
  """Day 12 pt1 solution."""
  ofst = ord("a")
  g = nx.DiGraph()
  lines = inp.splitlines()
  posns = set()
  heights = {}
  for idxh, ln in enumerate(lines):
    if idxh == 0:
      w = len(ln)
      h = len(lines)
    for idxw, node in enumerate(list(ln)):
      pos = tuple([idxw, idxh])
      if node == "S":
        height = ord("a") - ofst
      elif node == "E":
        height = ord("z") - ofst
        endpos = pos
      else:
        height = ord(node) - ofst
      heights[pos] = height
      g.add_node(pos, height=height, cell=node)
      posns.add(pos)
      pos_ab = tuple([idxw, idxh-1])
      pos_lf = tuple([idxw-1, idxh])
      if pos_ab in posns:
        h1 = heights[pos_ab]
        h2 = height
        log.trace(f"add-edges above {pos_ab}[{h1}] <-> {pos}[{h2}]")
        if h2 <= h1 + 1:
          log.trace(f"add-edge above {pos_ab} -> {pos}")
          g.add_edge(pos_ab, pos)
        if h1 <= h2 + 1:
          log.trace(f"add-edge above {pos_ab} <- {pos}")
          g.add_edge(pos, pos_ab)
      if pos_lf in posns:
        h1 = heights[pos_lf]
        h2 = height
        log.trace(f"add-edges above {pos_lf}[{h1}] <-> {pos}[{h2}]")
        if h2 <= h1 + 1:
          log.trace(f"add-edge above {pos_lf} -> {pos}")
          g.add_edge(pos_lf, pos)
        if h1 <= h2 + 1:
          log.trace(f"add-edge above {pos_lf} <- {pos}")
          g.add_edge(pos, pos_lf)
  log.trace(f"graph={g}")
  
  min_pthlen = 1e9
  for pos in posns:
    if heights[pos] > 0:
      continue
    log.debug(f"trying start-pos={pos} height={heights[pos]}")
    try:
      pth = nx.shortest_path(g, source=pos, target=endpos)
      pthlen = len(pth)-1  # dont count start node
    except:  # path not found / no
      pthlen = 1e9
    if pthlen < min_pthlen:
      min_pthlen = pthlen
      log.debug(f"new min-path-len={min_pthlen}")
  return min_pthlen

In [ ]:
part = "part 2"
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 29
result = solve_day12pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
log.setLevel(logging.INFO)
out = solve_day12pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 13: Distress Signal ---

In [ ]:
day = "Day 13"
symb = "day13"
part = "part 1"
tests = """
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
""".strip()

In [ ]:
def cmp(a, b):
  return (a > b) - (a < b) 

def check_order(lft, rgt):
  log.trace(f"check_order({lft}, {rgt})")
  if isinstance(lft, int) and isinstance(rgt, int):
    c = -1 * cmp(lft, rgt)
    log.trace(f"  i2i result={c}")
    return c
  elif isinstance(lft, list) and isinstance(rgt, int):
    rgt = [rgt]
    log.trace(f"  promote2r: {lft}, {rgt}")
  elif isinstance(lft, int) and isinstance(rgt, list):
    lft = [lft]
    log.trace(f"  promote2l: {lft}, {rgt}")
  if isinstance(lft, list) and isinstance(rgt, list):
    log.trace(f"  lst2lst {lft}, {rgt}")
    if len(lft) == 0 and len(rgt) == 0:
      log.trace(f"  lstlen ret=0")
      return 0
    elif len(lft) == 0:
      log.trace(f"  lstlen ret=1")
      return 1
    elif len(rgt) == 0:
      log.trace(f"  lstlen ret=-1")
      return -1
    else:
      lit, rit = lft[0], rgt[0]
      log.trace(f"  popped: lit={lit}, rit={rit}")
      frontrc = check_order(lit, rit)
      if frontrc == 0:
        return check_order(lft[1:], rgt[1:])
      else:
        return frontrc
  else:
    log.trace(f"  unexpected {lft}, {rgt}")
    

def solve_day13pt1(inp):
  """Day 13 pt1 solution."""
  pairs = []
  for pair in inp.split("\n\n"):
    #log.trace(f"found pair:\n{pair}")
    lst = mapl(eval, pair.splitlines())
    log.trace(f"found pair-items: {lst}")
    pairs.append(lst)
  ok_items = []
  for pairidx, pair in enumerate(pairs):
    log.trace(f"pair #{pairidx}: {pair}")
    lft, rgt = pair
    rc = check_order(lft, rgt)
    if rc == 1:
      ok_items.append(pairidx+1)
  return sum(ok_items)

In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.DEBUG)
expected = 13
result = solve_day13pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
log.setLevel(logging.INFO)
out = solve_day13pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
import functools
#sorted(words, key=cmp_to_key(strcoll))

def solve_day13pt2(inp):
  """Day 13 pt2 solution."""
  lines = []
  for line in inp.splitlines():
    line = line.strip()
    if line == "":
       continue
    ls = eval(line)
    log.debug(f"found line={ls}")
    lines.append(ls)
  log.debug(f"found total lines#={len(lines)}")
  divd1, divd2 = [[2]], [[6]]
  lines.append(divd1)
  lines.append(divd2)
  lines = sorted(lines, key=functools.cmp_to_key(check_order), reverse=True)
  log.debug(f"lines="); mapl(lambda it: log.debug(f"  {it}"), lines)
  idx1, idx2 = lines.index(divd1)+1, lines.index(divd2)+1
  return idx1*idx2

In [ ]:
part = "part 2"
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 140
result = solve_day13pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
log.setLevel(logging.INFO)
out = solve_day13pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 14: Regolith Reservoir ---

In [ ]:
day = "Day 14"
symb = "day14"
part = "part 1"
tests = """
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
""".strip()

In [ ]:
def read_grid(inp):
  grid = set()
  for line in inp.splitlines():
    pts = mapl(lambda it: mapl(int, it.split(",")), line.split(" -> "))
    log.trace(f"ln pts={pts}")
    for idx, ept in enumerate(pts):
      if idx == 0:
        continue
      spt = pts[idx-1]
      if spt[0] == ept[0]: # wall along x
        if not spt[1] <= ept[1]:
          spt, ept = ept, spt
        x = spt[0]
        for y in range(spt[1], ept[1]+1):
          log.trace(f"add wall-block {tuple([x, y])}")
          grid.add(tuple([x, y]))
      elif spt[1] == ept[1]: # wall along y
        if not spt[0] <= ept[0]:
          spt, ept = ept, spt
        y = spt[1]
        for x in range(spt[0], ept[0]+1):
          log.trace(f"add wall-block {tuple([x, y])}")
          grid.add(tuple([x, y]))
      else:
        assert False, "no straight wall"
  return grid

def get_place(grid, ymx):
  pos =  tuple([500, 0])
  #log.trace(f"init-pos={pos} grid={grid}")
  while True:
    if pos[1] > ymx:
      log.debug(f"fallen pos={pos}")
      return None
    pt_s, pt_sw, pt_se = tuple([pos[0], pos[1]+1]), tuple([pos[0]-1, pos[1]+1]), tuple([pos[0]+1, pos[1]+1])
    if pt_s in grid and pt_sw in grid and pt_se in grid:
      log.trace(f"blocked rest-pos={pos}")
      return(pos)
    elif not pt_s in grid:
      pos = pt_s
      log.trace(f"move-S  pos={pos}")
    elif not pt_sw in grid:
      pos = pt_sw
      log.trace(f"move-SW pos={pos}")
    elif not pt_se in grid:
      pos = pt_se
      log.trace(f"move-SE pos={pos}")
    else:
      assert False, "unexpected"

def solve_day14pt1(inp):
  """Day 14 pt1 solution."""
  grid = read_grid(inp)
  ymx = max(mapl(lambda it: it[1], grid))
  log.trace(f"grid max_y={ymx}")
  log.trace(f"grid={grid}")
  ct = 0
  while True:
    pt = get_place(grid, ymx)
    if pt is None:
      log.info(f"fallen grain, short-circuit ct={ct}")
      break
    grid.add(pt)
    ct += 1
    log.debug(f"ct={ct} grain rests on pt={pt}")
    if ct % 1000 == 0:
      log.info(f"grain-rest-count={ct}")
  return ct

In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.WARN)
expected = 24
result = solve_day14pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
#log.setLevel(aoc.LOGLEVEL_TRACE)
#log.setLevel(logging.WARN)
out = solve_day14pt1(ins)
f"{day} {part} result: {out}" |> print

In [ ]:
def solve_day14pt2(inp):
  """Day 14 pt2 solution."""
  grid = read_grid(inp)
  ykeys = mapl(lambda it: it[1], grid)
  ymin, ymax = min(ykeys), max(ykeys)
  log.debug(f"grid max_y={ymax}, wall-pts-#={len(grid)}")
  log.trace(f"grid={grid}")

  ## add floor:
  ymax += 2
  yspan = ymax - ymin
  log.debug(f"yspan={yspan}")
  xkeys = mapl(lambda it: it[0], grid)
  xmin, xmax = min(xkeys)-2*yspan-2, max(xkeys)+2*yspan-2
  for x in range(xmin, xmax+1):
    log.trace(f"add wall-block {tuple([x, ymax])}")
    grid.add(tuple([x, ymax]))
  log.debug(f"added floor @max_y={ymax} x={[xmin, xmax]}")

  ct = 0
  while True:
    pt = get_place(grid, ymax)
    if pt is None:
      log.info(f"fallen,short-circuit ct={ct}")
      break
    grid.add(pt)
    ct += 1
    log.debug(f"ct={ct} grain rests on pt={pt}")
    if pt == tuple([500, 0]):
      log.info(f"grain max-filled ct={ct} pos={pt}")
      break
    if ct % 2000 == 0:
      log.info(f"grain-rest-count={ct}")
  return ct

In [ ]:
part = "part 2"
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
#log.setLevel(logging.INFO)
expected = 93
result = solve_day14pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
#log.setLevel(logging.WARN)
out = solve_day14pt2(ins)
f"{day} {part} result: {out}" |> print

### --- Day 15: Beacon Exclusion Zone ---

In [ ]:
day = "Day 15"
symb = "day15"
part = "part 1"
tests = """
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
""".strip()

In [ ]:
def read_input(inp):
  """Read input string into suitable data-structure."""
  # Sensor at x=2, y=18: closest beacon is at x=-2, y=15
  data = {}
  for line in inp.splitlines():
    #log.trace(f"line={line}")
    m = re.search(r"^Sensor at x=(.*): closest beacon is at x=(.*)", line)
    l = mapl(lambda it: mapl(int, it.replace(" y=","").split(",")), [m.group(1), m.group(2)])
    senspos = (l[0][0], l[0][1])
    beakpos = (l[1][0], l[1][1])
    data[senspos] = beakpos
    #log.trace(f"read sensor data: {l} {senspos}=>{beakpos}")
  return data

def get_manhattan_dist(a, b):
  """Return manhattan distance of two points a and b (2d)."""
  return abs(a[0]-b[0]) + abs(a[1]-b[1])

def get_manhattan_dist_points(a, dist, linenum=None):
  """Return all points within manhattan distance from a (2d).
  If linenum given only return points on line=linenum."""
  #log.trace(f"get_manhattan_dist_points({a}, {dist})")
  pts = set()
  if dist == 0:
    return []
  ptx, pty = a
  mdy = abs(pty-linenum)
  for y in range(pty-dist, pty+dist+1):
    if linenum is not None and y != linenum:
      continue
    for x in range(ptx-dist+mdy, ptx+dist-mdy+1):
      mdx = abs(x-ptx)
      pos = (x, y)
      #log.trace(f"  mark_pos={pos}")
      pts.add(pos)
  return pts

def solve_day15pt1(inp, linenum):
  """Day 15 pt1 solution."""
  data = read_input(inp)
  #log.trace(f"data={data}")
  log.trace(f"len_data={len(data.keys())}")
  
  wrld = {}
  for k, v in data.items():
    wrld[k] = "S"
    wrld[v] = "B"
  log.debug(f"world: keys_len={len(wrld.keys())}")

  for k,v in data.items():
    md = get_manhattan_dist(k, v)
    assert md > 0, "sensor beacon distance > 0"
    log.debug(f"data-item k={k} v={v} manht_dst={md}")
    for pt in get_manhattan_dist_points(k, md, linenum=linenum):
      if not pt in wrld.keys():
        wrld[pt] = "u"
  log.debug(f"world: mapped_len={len(wrld.keys())}")
  points_set = filterl(lambda it: it[1]==linenum, wrld.keys())
  log.debug(f"line={linenum} line-points: pts_len={len(points_set)}")
  #beacons_set = filterl(lambda k,v: , wrld.items())
  beacons_set = {k: v for k, v in wrld.items() if k[1]==linenum and v=="B"}
  log.debug(f"line={linenum} pts_len={len(points_set)} beacs={len(beacons_set)}")
  return len(points_set)-len(beacons_set)

In [ ]:
#get_manhattan_dist_points((1,1), 1) => 5
#get_manhattan_dist_points((2,2), 2)

In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.DEBUG)
expected = 26
result = solve_day15pt1(tests, linenum=10)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
#log.setLevel(aoc.LOGLEVEL_TRACE)
#log.setLevel(logging.WARN)
tm_start = int(time.time())
out = solve_day15pt1(ins, linenum=2000000)
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print

In [ ]:
def get_manhattan_dist_boundarypoints(a, dist, bounds):
  """Return the outer boundary points (x-y-tuples, 2d) from center a with distance dist,
  within coordinate boundaries bounds (min,max list)."""
  #log.trace(f"get_manhattan_dist_points({a}, {dist})")
  pts = set()
  if dist == 0:
    return []
  ptx, pty = a
  bd_min, bd_max = bounds
  for y in range(pty-dist, pty+dist+1):
    if y < bd_min or y > bd_max:
      continue
    mdy = abs(pty-y)
    for x in [ptx-dist+mdy, ptx+dist-mdy]:
      if x < bd_min or x > bd_max:
        continue
      pos = (x, y)
      pts.add(pos)
  return pts

def solve_day15pt2(inp, bounds):
  """Day 15 pt2 solution."""
  data = read_input(inp)
  log.debug(f"lsolve_day15pt2: en_data={len(data.keys())}, bounds={bounds}")
  
  wrld = {}
  for k, v in data.items():
    wrld[k] = "S"
    wrld[v] = "B"
  log.debug(f"world: keys_len={len(wrld.keys())}")

  # find the point having most intersections with beacon range boundaries:
  idx = -1
  points = defaultdict(int)
  for k, v in data.items():
    idx += 1
    md = get_manhattan_dist(k, v)
    assert md > 0, "sensor beacon distance > 0"
    log.debug(f"sensor data-item k={k} v={v} manht_dst={md}")
    if idx == 0:
      point_set = get_manhattan_dist_boundarypoints(k, md+1, bounds=bounds)
      for pt in point_set:
        points[pt] += 1
    else:
      point_set = get_manhattan_dist_boundarypoints(k, md+1, bounds=bounds)
      for pt in point_set:
        points[pt] += 1
  valmax = max(points.values())
  log.debug(f"boundpoints-len={len(points.keys())} valmax={valmax}")
  point_set = {k: v for k, v in points.items() if v==valmax}
  log.debug(f"boundpoints-max-len={len(point_set.keys())} points-max={point_set}")
  assert len(point_set) == 1
  point = list(point_set)[0]
  pt_val = point[0] * 4000000 + point[1]
  log.debug(f"found point={point} val={pt_val}")
  return pt_val

In [ ]:
#get_manhattan_dist_boundarypoints((2,2), 1, [0, 4])
#get_manhattan_dist_boundarypoints((2,2), 2, [0, 4])
#get_manhattan_dist((2,18), (-2,15)) |> print
#get_manhattan_dist((2,18), (14,11)) |> print

In [ ]:
part = "part 2"
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 56000011
result = solve_day15pt2(tests, bounds=[0, 20])
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
log.setLevel(logging.DEBUG)
tm_start = int(time.time())
out = solve_day15pt2(ins, bounds=[0, 4000000])
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print

### --- Day 16: Proboscidea Volcanium ---

In [ ]:
day = "Day 16"
symb = "day16"
part = "part 1"
tests = """
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
""".strip()

In [ ]:
assert set(["AA", "BB"]) == set(["BB", "AA"])

In [ ]:
import networkx as nx

class fxset(frozenset):
  def __repr__(self):
    s = super(fxset, self).__repr__()
    return s.replace("fxset({", "{!", 1).replace("})", "}", -1)

def read_input(inp):
  """Read input string into suitable data-structure."""
  data = {}
  nodes = set()
  edges = set()
  weights = {}
  actives = []
  seens = set()
  g = nx.Graph() # undirected Graph
  for line in inp.splitlines():
    #log.trace(f"read input line: {line}")
    m = re.search(r"^Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)", line)
    assert m is not None, f"unparsed line {line}"
    nm = m.group(1)
    fr = int(m.group(2))
    nxt = m.group(3).split(", ")
    #log.trace(f"found line id={nm}: fr={fr}. nxt={nxt}")
    #data[nm] = {"w": fr, "nxt": nxt}
    seen = False
    if nm == "AA":
      seens.add(nm)
      actives.append(nm)
    weights[nm] = fr
    #nodes[nm] = {"weight": fr, "seen": seen, "active": False}
    nodes.add(nm)
    #log.trace(f"add-node {nm}={nodes[nm]}")
    g.add_node(nm, weight=fr, seen=seen, active=False)
    for trg in nxt:
      log.trace(f"add-edge {set((nm, trg))}")
      edges.add(fxset((nm, trg)))
      g.add_edge(nm, trg)
  log.trace(f"wrld nodes: #={len(nodes)} {nodes}")
  log.trace(f"  edges: #={len(edges)} {edges}")
  log.trace(f"  weigths: {weights}")
  weights = {k: v for k, v in sorted(weights.items(), key=lambda it: it[1], reverse=True) if v > 0 or k == "AA"}
  #beacons_set = filterl(lambda k,v: , wrld.items())
  #beacons_set = {k: v for k, v in wrld.items() if k[1]==linenum and v=="B"}
  #weights = filterl(weights)
  log.trace(f"  weigths: {weights}")
  log.trace(f"  actives: {actives}")
  log.trace(f"  seens: {seens}")
  paths = {}
  for pair in itertools.combinations(weights.keys(), 2):
    pln = nx.shortest_path_length(g, pair[0], pair[1])
    log.trace(f"pair: {pair}, SPlen={pln} SPlen={nx.shortest_path(g, pair[0], pair[1])}")
    paths[fxset(pair)] = pln
  data["weights"] = weights
  data["paths"] = paths
  data["actives"] = actives
  return data

def fpr(o):
  return str(o).replace(" ", "").replace("'","")

MAXITER = 1_000_000
gct = 0
seen = set()
maxscore = -1

def iter_graph(tm, pos, gen, score, actives, steps, weights, paths):
  assert tm >= 0
  global gct, seen, maxscore
  gct += 1
  assert gct <= MAXITER, "MAXLOOP reached"
  #isprobe = "".join(actives) in "".join(["AA", "DD", "BB", "JJ", "HH", "EE", "CC"])
  isprobe = False
  if isprobe:
    lvl = logging.INFO
  else:
    lvl = logging.DEBUG
  log.log(lvl, fpr(f"{gct}:iter_graph(tm={tm}, pos={pos}, gen={gen}, score={score}, actives={actives}), steps={steps}, ...)"))
  ast = ",".join(actives)
  assert ast not in seen, f"path {ast} already seen"
  seen.add(ast)
  lpaths = [it for it in paths.items() if pos in it[0] and it[1] < tm]
  lpaths = [it for it in lpaths if not (list(it[0])[0] in actives and list(it[0])[1] in actives)]
  log.trace(f"lpaths2={fpr(lpaths)}")
  if score > maxscore:
    log.info(f"{gct}:new1 highscore={maxscore}, tm={tm}, gen={gen} for {fpr(actives)}")
    maxscore = score
  if len(lpaths) == 0:
    log.log(lvl, f"no more paths, tm={tm}, gen={gen}, score={score}, {fpr(actives)}")
    asc = tm * gen
    score += asc
    log.log(lvl, f"playdown {pos}, tm={tm}, gen={gen}, asc={asc}, score={score}<-{score-asc}, {fpr(actives)}")
    tm = 0
    if score > maxscore:
      maxscore = score
      log.info(f"{gct}:new2 highscore={maxscore}, tm={tm}, gen={gen} for {fpr(actives)}")
    return
  for k, v in lpaths:
    if list(k)[0] == pos:
      nxtpos = list(k)[1]
    else:
      nxtpos = list(k)[0]
    assert not nxtpos in actives, f"nxtpos={nxtpos} from {pos} in actives={actives}"
    log.trace(f"  path goto={nxtpos} from {pos}; fromsteps={steps}")
    iterated = True
    nxtsteps = steps.copy()
    nxtsteps.append(nxtpos)

    log.trace(f"  activate pos={nxtpos}, wt={weights[nxtpos]}, gen={gen}, fromsteps={fpr(steps)}")
    nxtactives = actives.copy()
    nxtactives.append(nxtpos)
    nxtsteps.append("*")
    nxtgen = gen + weights[nxtpos]
    nxttm = tm - v - 1
    nxtscore = score + gen * (v+1)
    log.log(lvl, f"switchon {nxtpos}, tm={nxttm}<-{tm}, gen={nxtgen}<-{gen}, score={nxtscore}<-{score}, {fpr(actives)}")
    if nxttm >= 0:
      iter_graph(nxttm, nxtpos, nxtgen, nxtscore, nxtactives, nxtsteps, weights, paths)
    else:
      log.info(f"endconf {fpr(actives)} steplen={len(steps)} steps={fpr(steps)}")


def solve_day16pt1(inp):
  """Day 16 pt1 solution."""
  global gct, maxscore
  gct = 0
  maxcscore = -1
  data = read_input(inp)
  #log.trace(f"len_data={len(data.keys())}")
  log.debug(f"data={data}")
  log.info(f"data #={len(data['weights'].keys())} weights={data['weights']}")
  iter_graph(30, "AA", 0, 0, data["actives"], [], data["weights"], data["paths"])
  log.info(f"gct={gct}")
  return maxscore


In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 1651
result = solve_day16pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
tm_start = int(time.time())
out = solve_day16pt1(ins)
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print

In [ ]:
def solve_day16pt2(inp, bounds):
  """Day 16 pt2 solution."""
  return -1

In [ ]:
part = "part 2"
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = -1
result = solve_day15pt2(tests, bounds=[0, 20])
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
log.setLevel(logging.DEBUG)
tm_start = int(time.time())
out = solve_day15pt2(ins, bounds=[0, 4000000])
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print

### --- Day 17: Pyroclastic Flow ---

In [ ]:
day = "Day 17"
symb = "day17"
part = "part 1"
test1 = """
####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##
""".strip()

test2 = """
>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
""".strip()

In [ ]:
def read_shapes(inp):
  return mapl(lambda it: it.strip(), inp.split("\n\n"))

def refn_shape(shape_str):
  l = mapl(lambda ln: mapl(int, list(ln.replace("#","1").replace(".","0"))), shape_str.split("\n"))
  return list(reversed(l))

def tsps_shape(shape, n):
  outshape = []
  for idx, row in enumerate(shape):
    if n > 0:
      outshape.append([0 for i in range(n)] + row.copy())
    elif n < 0:
      outshape.append(row[abs(n):].copy())
  log.trace(f"tsps_shape(shp, {n})={outshape}")
  return outshape

def read_input(inp):
  """Read input string into suitable data-structure."""
  return list(inp)

def get_shape_breadths(shapes):
  shape_breadths = []
  for shape in shapes:
    shape = shape.replace(".", "")
    shape_breadths.append(max(mapl(len, shape.split("\n"))))
  return shape_breadths

def reprwrld(wrld):
  s = ""
  for ln in reversed(wrld):
    #s += "".join(mapl(lambda it: str(it).replace("9", "X"), ln)) + "\n"
    s += "".join(mapl(lambda it: str(it), ln)) + "\n"
  return s.strip()

def shift_wrld(wrld, wind, report=True):
  wrld = copy.deepcopy(wrld)
  if wind == ">":
    if "19" in reprwrld(wrld):
      return [wrld, True] 
    else:
      for idx, ln in enumerate(wrld):
        if not 1 in ln:
          continue
        ln2 = "".join(mapl(str,ln))
        #log.trace(f"ln2={ln2}")
        ln2 = re.sub(r'([1]+)0', r'0\1', ln2)
        ln = mapl(int, list(ln2))
        #log.trace(f":> {ln}")
        wrld[idx] = ln
  elif wind == "<":
    if "91" in reprwrld(wrld):
      return [wrld, True] 
    else:
      for idx, ln in enumerate(wrld):
        if not 1 in ln:
          continue
        ln2 = "".join(mapl(str,ln))
        #log.trace(f"ln2={ln2}")
        ln2 = re.sub(r'(0)([1]+)', r'\2\1', ln2)
        ln = mapl(int, list(ln2))
        #log.trace(f":< {ln}")
        wrld[idx] = ln
  if report:
    log.debug(f"post wrld=\n{reprwrld(wrld)}")
  return [wrld, False]
    
def drop_wrld(wrld):
  wrld = copy.deepcopy(wrld)
  wrld_t = transpose_lol(wrld)
  #log.trace(reprwrld(reversed(wrld_t)))
  wrld2, bounce = shift_wrld(wrld_t, "<", report=False)
  #log.trace(f"Tshift: clsn={bounce}\n{wrld2}")
  if not bounce:
    wrld3 = transpose_lol(wrld2)
    #log.trace(f"TreTsps:\n{wrld3}")
    #log.trace(f":::\n{reprwrld(wrld3)}")
    if wrld3[-1] == list(mapl(int, "900000009")):
      wrld3 = wrld3[0:len(wrld3)-1]
    wrld = wrld3
  collision = bounce
  if collision:
    for rowidx in range(len(wrld)):
      for colidx in range(len(wrld[0])):
        if wrld[rowidx][colidx] == 1:
          wrld[rowidx][colidx] = 9
  log.debug(f"drop_wrld(wrld) clsn={collision}\n{reprwrld(wrld)}")
  return [wrld, collision]

def solve_day17pt1(inp, shapes, iterations):
  """Day 16 pt1 solution."""
  initpt = (1, -4)  # 3 empty rows below
  voidline = list(mapl(int, "900000009"))
  florline = list(mapl(int, "999999999"))
  wrld = [florline]
  grid = [voidline]
  log.trace(f"inipt={initpt}, grid={grid}")
  data = read_input(inp)
  shapelen = len(shapes)
  windlen = len(data)
  log.trace(f"shapelen={shapelen} windlen={len(data)}")
  shape_breadths = get_shape_breadths(shapes)
  shape_heights = mapl(lambda shp: len(shp.split("\n")), shapes)
  log.trace(f"shape_breadths={shape_breadths}; shape_heights={shape_heights}")

  shapenum = 1
  shapeidx = 0
  shape = shapes[shapeidx]
  shape_brdt = shape_breadths[shapeidx]
  shaperefd = refn_shape(shape)
  log.debug(f"shp :\n{shape}")
  log.debug(f"shp2:\n{shaperefd}")
  wrld.append(voidline)
  wrld.append(voidline)
  wrld.append(voidline)
  shapeins = tsps_shape(shaperefd, 2)
  log.debug(f"shp-int:\n{shapeins}")
  for ln in shapeins:
    ln2 = [9] + ln + [0 for i in range(7-len(ln))] + [9]
    wrld.append(ln2)
  log.debug(f"wrld#in:\n{reprwrld(wrld)}")
  
  stp = 0
  tms = int( time.time() )
  while True:
    stp += 1
    windidx = (stp-1) % windlen
    wind = data[windidx]
    log.debug(f"stp#{stp} shp#{shapeidx}_{shape_brdt} wind#{windidx}={wind}")
    log.debug(f"wrld#{stp}:\n{reprwrld(wrld)}")
    wrld, bounce = shift_wrld(wrld, wind)
    wrld, collision = drop_wrld(wrld)
    if collision:
      #log.info(f"stp#{stp} shape#={shapenum} clsn! height={len(wrld)-1}\n{reprwrld(wrld)}")
      if shapenum % 100 == 0:
        tmd = int( time.time() ) - tms
        log.info(f"stp#{stp} shape#={shapenum} clsn! height={len(wrld)-1}. took {tmd} secs")
      if shapenum >= iterations:
        break
      shapeidx = shapenum % shapelen
      shapenum += 1
      shape = shapes[shapeidx]
      shape_brdt = shape_breadths[shapeidx]
      shaperefd = refn_shape(shape)
      log.debug(f"stp#{stp} collision! new-shape=\n{shaperefd}")
      shapeins = tsps_shape(shaperefd, 2)
      log.debug(f"shp-int:\n{shapeins}")
      wrld.append(voidline)
      wrld.append(voidline)
      wrld.append(voidline)
      for ln in shapeins:
        ln2 = [9] + ln + [0 for i in range(7-len(ln))] + [9]
        wrld.append(ln2)
      log.debug(f"wrld#in:\n{reprwrld(wrld)}")
  return len(wrld)-1

In [ ]:
f"{day} {part}" |> print
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
expected = 3068
iterations = 2022
shapes = read_shapes(test1)
log.trace(f"shapes={shapes}")

#result = solve_day17pt1(test2, shapes=shapes, iterations=5)

result = solve_day17pt1(test2, shapes=shapes, iterations=iterations)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
result = solve_day17pt1(ins, shapes=shapes, iterations=iterations)
f"{day} {part} result: {result}" |> print

In [ ]:
f"{day} {part} result: {result}" |> print

### --- Day 18: Boiling Boulders ---

In [ ]:
day = "Day 18"
symb = "day18"
part = "part 1"
tests = """
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
""".strip()

In [ ]:
def read_input(inp):
  """Read input string into suitable data-structure."""
  lst = []
  for ln in inp.splitlines():
    t = mapl(int, ln.split(","))
    lst.append( tuple(t) )
  log.trace(f"read_input() out={lst}")
  return lst

def num_neighbors(pt, data):
  x, y, z = pt
  pn = [
    (x-1, y, z), (x+1, y, z),
    (x, y-1, z), (x, y+1, z),
    (x, y, z-1), (x, y, z+1)
  ]
  neibs = 0
  for npt in pn:
    if npt in data:
      neibs += 1
  return neibs

def solve_day18pt1(inp):
  """Day 18 pt1 solution."""
  data = read_input(inp)
  free_surf_num = 0
  for pt in data:
    free_surf_num_pt = 6 - num_neighbors(pt, data)
    free_surf_num += free_surf_num_pt
  return free_surf_num

In [ ]:
f"{day} {part}" |> print
log.setLevel(aoc.LOGLEVEL_TRACE)
#log.setLevel(logging.INFO)
expected = 64
result = solve_day18pt1(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
ins = aoc.read_file_to_str(f"./in/{symb}.in").strip()
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.INFO)
tm_start = int(time.time())
out = solve_day18pt1(ins)
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print

In [ ]:
def count_enclosed(data):
  xs = [pt[0] for pt in data]
  ys = [pt[1] for pt in data]
  zs = [pt[2] for pt in data]
  minx, maxx = min(xs), max(xs)
  miny, maxy = min(ys), max(ys)
  minz, maxz = min(zs), max(zs)
  log.debug(f"x:[{minx}..{maxx}], y:[{miny}..{maxy}], z:[{minz}..{maxz}]")
  num_enclosed = 0
  num_empties = 0
  for x in range(minx+1, maxx):
    for y in range(miny+1, maxy):
      for z in range(minz+1, maxz):
        #log.trace(f"scan {(x, y, z)}")
        num_empties += 1
        if not (x, y, z) in data:
          #log.trace(f"{(x, y, z)} is empty")
          if num_neighbors((x, y, z), data) == 6:
            num_enclosed += 1
            #log.trace(f"{(x, y, z)} is enclosed")
  log.debug(f"count_enclosed(d): num_empties={num_empties}, num_enclosed_cells={num_enclosed}")
  return num_enclosed

def solve_day18pt2(inp):
  """Day 18 pt2 solution."""
  data = read_input(inp)
  free_surf_num = 0
  for pt in data:
    free_surf_num_pt = 6 - num_neighbors(pt, data)
    free_surf_num += free_surf_num_pt
  n = count_enclosed(data)
  log.debug(f"found enclosed-#={n}")  
  free_surf_num2 = free_surf_num - 6 * n
  return free_surf_num2

In [ ]:
f"{day} {part}" |> print
log.setLevel(aoc.LOGLEVEL_TRACE)
#log.setLevel(logging.INFO)
expected = 58
result = solve_day18pt2(tests)
aoc.assert_msg(f"test solve {day} {part} expected={expected} result={result}", result == expected)

In [ ]:
#log.setLevel(aoc.LOGLEVEL_TRACE)
log.setLevel(logging.DEBUG)
tm_start = int(time.time())
out = solve_day18pt2(ins)
tm_delta = int(time.time()) - tm_start
f"{day} {part} result: {out}, took {tm_delta} secs" |> print